In [20]:
import os
import pickle
import numpy as np
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import gym
from optical_rl_gym.utils import evaluate_heuristic, random_policy
# from optical_rl_gym.envs.rwa_env_focs_v2 import kSP_FF, FF_kSP, kSP_MU, CA_MU
from optical_rl_gym.heuristics import kSP_FF, FF_kSP, kSP_MU, CA_MU
from optical_rl_gym.envs.rwa_env_focs_v2 import PathOnlyFirstFitAction

In [2]:
def print_service_stats(env):
    print("Whole training process statistics:")
    rnd_path_action_probability = np.sum(env.actions_output, axis=1) / np.sum(env.actions_output)
    rnd_wavelength_action_probability = np.sum(env.actions_output, axis=0) / np.sum(env.actions_output)
    print('Path action probability:', np.sum(env.actions_output, axis=1) / np.sum(env.actions_output))
    print('Wavelength action probability:', np.sum(env.actions_output, axis=0) / np.sum(env.actions_output))
    num_lps_reused = env.num_lightpaths_reused
    print('Load (Erlangs):', load)
    print('Service bit rate (Gb/s):', env.service.bit_rate/1e9)
    print('Total number of services:', env.services_processed)
    print('Total number of accepted services:', env.services_accepted)
    print('Proportion of services provisioned:', env.services_accepted/env.services_processed)
    print('Number of services on existing lightpaths:', num_lps_reused)
    print('Number of services released:', env.num_lightpaths_released)
    print('Number of transmitters on each node:', env.num_transmitters)
    print('Number of receivers on each node:', env.num_receivers)
    print('Final throughput (TB/s):', env.get_throughput()/1e12)
def get_service_utils(env):
    path_id_util = []
    num_paths = env.topology.number_of_nodes() * (env.topology.number_of_nodes() - 1) * env.k_paths
    for id in range(num_paths):
        path_id_util.append(np.sum(env.lightpath_service_allocation[id,:]))
    return path_id_util, num_paths

In [3]:
# with open('/Users/joshnevin/RL_FOCSLab/topologies/3_node_network_sym.h5', 'rb') as f:
#     topology = pickle.load(f)
# node_request_probabilities = np.array([0.333333,0.333333,0.333333])

In [4]:
with open('/Users/joshnevin/RL_FOCSLab/topologies/nsfnet_chen_5-paths_directional.h5', 'rb') as f:
    topology = pickle.load(f)
# node probabilities from https://github.com/xiaoliangchenUCD/DeepRMSA/blob/6708e9a023df1ec05bfdc77804b6829e33cacfe4/Deep_RMSA_A3C.py#L77
node_request_probabilities = np.array([0.01801802, 0.04004004, 0.05305305, 0.01901902, 0.04504505,
       0.02402402, 0.06706707, 0.08908909, 0.13813814, 0.12212212,
       0.07607608, 0.12012012, 0.01901902, 0.16916917])

In [21]:
load = int(1e10)
env_args = dict(topology=topology, seed=5, load = load,
                allow_rejection=False, # the agent cannot proactively reject a request
                mean_service_holding_time=1e8,
                episode_length=1500, node_request_probabilities=node_request_probabilities, exp_request_res = 25e9,
               exp_request_lambda = 1, term_on_first_block=True)
env_0 = PathOnlyFirstFitAction(gym.make('RWAFOCS-v2', **env_args))
env_1 = gym.make('RWAFOCS-v2', **env_args)
env_2 = gym.make('RWAFOCS-v2', **env_args)
env_3 = gym.make('RWAFOCS-v2', **env_args)
env_4 = gym.make('RWAFOCS-v2', **env_args)

In [23]:
mean_reward, std_reward = evaluate_heuristic(env_0, random_policy, n_eval_episodes=30,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 348.1333333333333
Std reward: 203.2385680808531


In [24]:
mean_reward + std_reward

551.3719014141864

In [7]:
heuristic = kSP_FF
mean_reward, std_reward = evaluate_heuristic(env_1, kSP_FF, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1461.0
Std reward: 0.0


In [8]:
mean_reward, std_reward = evaluate_heuristic(env_2, FF_kSP, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1463.0
Std reward: 0.0


In [13]:
mean_reward, std_reward = evaluate_heuristic(env_3, CA_MU, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1466.0
Std reward: 0.0


In [14]:
mean_reward, std_reward = evaluate_heuristic(env_4, kSP_MU, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1465.0
Std reward: 0.0


In [15]:
print_service_stats(env_0)

Whole training process statistics:
Path action probability: [0.16666667 0.16666667 0.16666667 0.22222222 0.27777778]
Wavelength action probability: [0.         0.         0.         0.         0.05555556 0.
 0.05555556 0.         0.         0.05555556 0.         0.
 0.         0.05555556 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.05555556 0.         0.         0.         0.
 0.         0.05555556 0.05555556 0.         0.05555556 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.05555556 0.         0.         0.05555556
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.05555556 0.         0.         0.         0.05555556 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.05555556
 0.         0.05555556 0.         0.         0.         0.
 0.       

kSP-FF:

In [16]:
print_service_stats(env_1)

Whole training process statistics:
Path action probability: [0.63997262 0.19575633 0.08761123 0.04175222 0.0349076 ]
Wavelength action probability: [0.0109514  0.00958248 0.0109514  0.01163587 0.01232033 0.0109514
 0.01232033 0.01026694 0.00958248 0.00958248 0.01026694 0.01026694
 0.01026694 0.01232033 0.01232033 0.0109514  0.01232033 0.00752909
 0.00821355 0.00889802 0.00889802 0.00958248 0.01300479 0.01163587
 0.00889802 0.00889802 0.01026694 0.00889802 0.00958248 0.01232033
 0.00889802 0.01026694 0.0109514  0.01026694 0.00752909 0.00889802
 0.0109514  0.00958248 0.01026694 0.01163587 0.00821355 0.00889802
 0.00958248 0.01232033 0.0109514  0.01026694 0.00821355 0.01026694
 0.0109514  0.00821355 0.00889802 0.00821355 0.00958248 0.01232033
 0.00821355 0.00958248 0.01163587 0.00958248 0.01163587 0.01163587
 0.00821355 0.00684463 0.00821355 0.00958248 0.00889802 0.01232033
 0.01232033 0.00958248 0.00889802 0.00958248 0.0109514  0.01163587
 0.01026694 0.00821355 0.00889802 0.01232033 0.01

FF-kSP:

In [17]:
print_service_stats(env_2)

Whole training process statistics:
Path action probability: [0.54272044 0.21804511 0.09979494 0.08065619 0.05878332]
Wavelength action probability: [0.01367054 0.01093643 0.0102529  0.0102529  0.01230349 0.01230349
 0.01093643 0.00956938 0.01093643 0.00615174 0.01093643 0.01093643
 0.01093643 0.01093643 0.01161996 0.0102529  0.01093643 0.01093643
 0.0075188  0.0075188  0.00956938 0.00820232 0.00820232 0.01161996
 0.01230349 0.0102529  0.00956938 0.00956938 0.01093643 0.0102529
 0.00956938 0.00888585 0.00956938 0.0102529  0.01230349 0.0102529
 0.0102529  0.00888585 0.0102529  0.00888585 0.00888585 0.01093643
 0.0102529  0.00956938 0.00956938 0.00956938 0.01161996 0.01093643
 0.00888585 0.0075188  0.00956938 0.00820232 0.0102529  0.01367054
 0.00888585 0.01230349 0.00888585 0.00888585 0.00956938 0.01161996
 0.00683527 0.00820232 0.00888585 0.00888585 0.01161996 0.0102529
 0.0102529  0.01161996 0.01230349 0.00956938 0.00888585 0.00956938
 0.01161996 0.0102529  0.00956938 0.0102529  0.0129

CA-MU:

In [18]:
print_service_stats(env_3)

Whole training process statistics:
Path action probability: [0.91473397 0.0361528  0.01841746 0.01432469 0.01637108]
Wavelength action probability: [0.00613915 0.0095498  0.01091405 0.01091405 0.01159618 0.01159618
 0.01296044 0.01227831 0.00886767 0.00886767 0.01023192 0.01023192
 0.01023192 0.01227831 0.01091405 0.01091405 0.0095498  0.01023192
 0.0095498  0.0095498  0.00613915 0.01023192 0.01023192 0.01091405
 0.01159618 0.00750341 0.01023192 0.01364256 0.01091405 0.01023192
 0.00750341 0.00886767 0.01023192 0.0095498  0.00886767 0.0095498
 0.01296044 0.01023192 0.01023192 0.00886767 0.00750341 0.00886767
 0.01091405 0.00886767 0.01023192 0.01296044 0.01091405 0.0095498
 0.01091405 0.01023192 0.00818554 0.00750341 0.01091405 0.01023192
 0.01023192 0.01023192 0.01023192 0.01023192 0.00886767 0.00886767
 0.00750341 0.00886767 0.01091405 0.00818554 0.01091405 0.01091405
 0.01364256 0.01023192 0.00886767 0.0095498  0.01091405 0.0095498
 0.00886767 0.01091405 0.00886767 0.01023192 0.0122

kSP-MU:

In [19]:
print_service_stats(env_4)

Whole training process statistics:
Path action probability: [0.95358362 0.02730375 0.00955631 0.00546075 0.00409556]
Wavelength action probability: [0.0109215  0.0116041  0.00955631 0.01296928 0.01023891 0.01296928
 0.0116041  0.01023891 0.00955631 0.00682594 0.01228669 0.01023891
 0.0116041  0.0109215  0.01365188 0.0109215  0.00819113 0.00819113
 0.00955631 0.00750853 0.00750853 0.01023891 0.01023891 0.0116041
 0.01023891 0.00819113 0.01228669 0.01023891 0.0109215  0.00819113
 0.0109215  0.00887372 0.01023891 0.00955631 0.00887372 0.01023891
 0.0116041  0.00887372 0.0109215  0.00887372 0.01023891 0.01023891
 0.00819113 0.01023891 0.0109215  0.00955631 0.0109215  0.01023891
 0.00955631 0.00955631 0.00887372 0.0109215  0.00955631 0.0109215
 0.00955631 0.01023891 0.0109215  0.00887372 0.00682594 0.01023891
 0.00682594 0.0116041  0.00819113 0.0116041  0.0109215  0.01023891
 0.0116041  0.0116041  0.01023891 0.00887372 0.0116041  0.00887372
 0.00819113 0.01023891 0.01023891 0.01228669 0.010

In [ ]:
path_id_util_1, num_paths = get_service_utils(env_1)
path_id_util_2, _ = get_service_utils(env_2)
path_id_util_3, _ = get_service_utils(env_3)
path_id_util_4, _ = get_service_utils(env_4)

In [ ]:
sum(path_id_util_1)

In [ ]:
sum(path_id_util_2)

In [ ]:
sum(path_id_util_3)

In [ ]:
sum(path_id_util_4)

Verify that throughput is the same, despite the number of allocated requests being different!

In [ ]:
req_comp = np.array([env_1.cumulative_throughput[i] - env_2.cumulative_throughput[i] for i in range(len(env_1.cumulative_throughput))])

In [ ]:
np.count_nonzero(req_comp)

In [ ]:
print(req_comp[-1])
print(req_comp[-2])
print(req_comp[-3])
print(req_comp[-4])
print(req_comp[-5])
print(req_comp[-6])
print(req_comp[-7])

In [ ]:
print(env_2.cumulative_throughput[-1])
print(env_2.cumulative_throughput[-2])

In [ ]:
plt.plot(env_1.cumulative_throughput, '+')
plt.show()

In [ ]:
plt.plot(env_2.cumulative_throughput, '+')
plt.show()

In [ ]:
plt.plot(req_comp)
plt.show()

In [ ]:
np.sum(env_2.active_bitrates)

In [ ]:
env_2.cumulative_throughput

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_1)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_kspff.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_2)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_3)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_4)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_1, 'r+')
plt.plot(x, path_id_util_2, 'bx')
plt.xlabel("Path ID")
plt.ylabel("Number of services")
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, abs(np.array(path_id_util_1) - np.array(path_id_util_2)))
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
(np.array(path_id_util_1) - np.array(path_id_util_2))

In [ ]:
np.array(path_id_util_1)

In [ ]:
np.array(path_id_util_2)

Look at ksp ordering:

In [ ]:
env_1.service.source

In [ ]:
paths_test = env_1.topology.graph['ksp']['2','3']
len(paths_test)

In [ ]:
paths_test[0].length

In [ ]:
paths_test[1].length

In [ ]:
paths_test[2].length

In [ ]:
paths_test[3].length

In [ ]:
paths_test[4].length